In [ ]:
import glob, pickle
import pathlib, os

import numpy as np
import scipy as sp
import pandas as pd

from scipy.interpolate import make_lsq_spline
from scipy.optimize import curve_fit
from scipy.interpolate import BSpline
import open3d as o3d
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
def cvt_polar(x,y):
    """Convert to polar coordinate system
    x, y -> r, theta
    """
    r = np.sqrt(x**2+y**2)
    theta = np.sign(y)*np.arccos(x/np.sqrt(x**2+y**2))
    return r, theta

def generate_knots(start, end, n_interval, d):
    """Generate knot vector
    Parameters
    ==============
    start: float
    end: float
        start and end of knots
    n_interval: int
        number of interval
    d: int
        degree of B-spline
    
    Returns
    ==============
    knots: array, shape(n_interval+2*dim + 1)
        knot vector
    
    """
    step = (end-start)/n_interval
    knots_ = [i for i in np.arange(start, end+step, step)]
    knots_p = [knots_[0] for i in range(d)]
    knots_a = [knots_[-1] for i in range(d)]
    
    knots = np.array(knots_p + knots_ + knots_a)
    return knots

def generate_knots_circular_bspline(start, end, n_interval, d):
    
    step = (end-start)/n_interval
    knots_ = [i for i in np.arange(start, end+step, step)]
    knots_p = [knots_[i+len(knots_)-2] - (knots_[-1] - knots_[0]) for i in range(1-d, 1)]
    knots_a = [knots_[i-len(knots_)]+ (knots_[-1]-knots_[0]) for i in range(len(knots_)+1,len(knots_)+d+2)]
    
    knots = knots_p + knots_ + knots_a
    
    return np.array(knots)

degree = 3
knots = generate_knots_circular_bspline(-np.pi, np.pi, 16, 3)

def bspline_wrapper(theta, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18, c19):
    bspline = BSpline(knots, 
                      [c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18, c19, 0], 
                      degree, extrapolate="periodic")
    return bspline(theta)

def load_cfs(file_path):
    with open(file_path, "rb") as f:
        curve_fragments = pickle.load(f)
    return curve_fragments

def marge_cfs(curve_fragments, output_folder, output_name):
    os.makedirs(output_folder, exist_ok=True)
    
    N_INTERVAL = 16
    DEGREE = 3

    theta_list = []
    curve_list = []

    for i, cf in enumerate(curve_fragments):
        print(len(cf))
        coord_3d = np.concatenate(cf)

        pca = PCA(n_components=2)
        coord_2d = pca.fit_transform(coord_3d)

        _, theta = cvt_polar(coord_2d[:,0], coord_2d[:,1])
        #theta = np.array([t+2*np.pi if t < -np.pi/2 else t for t in theta])
        
        degree = 3
        knots = generate_knots_circular_bspline(-np.pi, np.pi, 16, 3)

        sort_flag = True
        theta_sorted = None
        coord_3d_sorted = None
        while sort_flag:
            theta_r = theta + (10**-8)*np.random.random(len(theta))
            idx_sorted = np.argsort(theta_r)
            theta_sorted = theta_r[idx_sorted]
            coord_3d_sorted = coord_3d[idx_sorted]

            if len(theta) == len(np.unique(theta_sorted)):
                sort_flag = False
                
        popt_x, _ = curve_fit(bspline_wrapper, theta_sorted, coord_3d_sorted[:,0], bounds=(-20,20), ftol=10**-7)
        popt_y, _ = curve_fit(bspline_wrapper, theta_sorted, coord_3d_sorted[:,1], bounds=(-20,20), ftol=10**-7)
        popt_z, _ = curve_fit(bspline_wrapper, theta_sorted, coord_3d_sorted[:,2], bounds=(-20,20), ftol=10**-7)

        theta_recon = np.linspace(-np.pi, np.pi, 360)
        coord_3d_recon = np.stack([
            bspline_wrapper(theta_recon, *popt_x), 
            bspline_wrapper(theta_recon, *popt_y), 
            bspline_wrapper(theta_recon, *popt_z), 
            ]).T
        theta_list.append(theta_recon)
        curve_list.append(coord_3d_recon)
    output_name = os.path.join(output_folder, output_name)
    with open(output_name, 'wb') as f:
        pickle.dump(curve_list, f)

In [ ]:
curve_fragments_file = "SUPPORTED_CURVE_FILE"
output_folder = "SPLINED_CURVE_FOLDER"
output_name = "SPLINED_CURVE_NAME"

In [ ]:
curve_fragments_file = "data/supported_curve/demo_supported_curves.pickle"
output_folder = "data/splined_curve"
output_name = "demo_splined_curve.pickle"


In [ ]:
curve_fragments = load_cfs(curve_fragments_file)
marge_cfs(curve_fragments, output_folder, output_name)